This is an accompanying notebook for the paper *AKLT-states as ZX-diagrams: diagrammatic reasoning for quantum states*, by East, van de Wetering, Chancellor and Grushin.

In this notebook we demonstrate how a 1D AKLT chain can be constructed and simplified inside PyZX.

In [ ]:
import sys; sys.path.append('../..') # If this notebook is run as part of the PyZX repository, this makes Python find the local copy
import random, math, os
import pyzx as zx # This notebook was created with pyzx version 0.6.3 in mind, newer version might be incompatible.
from fractions import Fraction
zx.settings.drawing_auto_hbox = False

First, we construct the spin-1 symmetriser, that projects the 4-dimensional subspace of two qubit wires down to the 3-dimensional symmetric subspace.

In [ ]:
# We load the diagram via its description as a tikz diagram
tikz_symmetrizer = r"""\begin{tikzpicture}[tikzfig]
	\begin{pgfonlayer}{nodelayer}
		\node [style=none] (129) at (0, 0) {};
		\node [style=none] (131) at (3.25, 0) {};
		\node [style=none] (132) at (3.25, -1.5) {};
		\node [style=Green Node] (147) at (0.5, 0) {};
		\node [style=Green Node] (148) at (0.5, -1.5) {};
		\node [style=Red Node] (151) at (0.5, -0.75) {};
		\node [style=H] (153) at (1.25, -0.75) {};
		\node [style=Green Node] (155) at (1.25, -0.25) {};
		\node [style=H] (158) at (2, -0.75) {};
		\node [style=Green Node] (159) at (2.75, -0.75) {};
		\node [style=Red Node] (161) at (2.75, 0) {};
		\node [style=Red Node] (162) at (2.75, -1.5) {};
		\node [style=none] (167) at (0, -1.5) {};
	\end{pgfonlayer}
	\begin{pgfonlayer}{edgelayer}
		\draw (129.center) to (147);
		\draw (131.center) to (161);
		\draw (161) to (147);
		\draw (147) to (151);
		\draw (151) to (148);
		\draw (148) to (162);
		\draw (162) to (132.center);
		\draw (151) to (153);
		\draw (153) to (158);
		\draw (158) to (159);
		\draw (162) to (159);
		\draw (159) to (161);
		\draw (155) to (153);
		\draw (167.center) to (148);
	\end{pgfonlayer}
\end{tikzpicture}
"""
symmetrizer = zx.Graph.from_tikz(tikz_symmetrizer)
symmetrizer.auto_detect_inputs() # Tell PyZX which boundary vertices are inputs and outputs
symmetrizer.scalar.add_power(-1) # Correct normalisation
zx.draw(symmetrizer)
zx.print_matrix(symmetrizer)

We similarly construct the singlet state.

In [ ]:
tikz_singlet = r"""\begin{tikzpicture}[tikzfig]
	\begin{pgfonlayer}{nodelayer}
		\node [style=none] (163) at (1, -1.5) {};
		\node [style=Green Node] (164) at (0, 0) {$\pi$};
		\node [style=Red Node] (165) at (0, -1.5) {$\pi$};
		\node [style=none] (166) at (1, 0) {};
	\end{pgfonlayer}
	\begin{pgfonlayer}{edgelayer}
		\draw (163.center) to (165);
		\draw (165) to (164);
		\draw (164) to (166.center);
	\end{pgfonlayer}
\end{tikzpicture}
"""
singlet = zx.Graph.from_tikz(tikz_singlet)
singlet.auto_detect_inputs()
zx.draw(singlet)
zx.print_matrix(singlet)

In order to construct the AKLT chain by composing diagrams, we also need a small diagram to represent an identity on 1 wire.

In [ ]:
ID = zx.generate.identity(1)
zx.draw(ID)

And now we can construct the AKLT chain:

In [ ]:
N = 5 # Number of spins
chain = symmetrizer
for i in range(N-1):
    chain = chain @ symmetrizer.translate(0,0.5) # Compose diagrams vertically
singlets = ID.copy()
for i in range(N-1):
    singlets = singlets @ singlet.translate(0,0.5) # translate() shifts the diagram vertically, to make them align properly
singlets = singlets @ ID.translate(0,0.5)
chain =  singlets + chain # Compose them horizontally in diagrammatic order
zx.draw(chain)

Let us now verify that this state has a non-zero string order parameter. This means that a state like $\lvert +1,0,0,0,+1 \rangle$ is not part of the superposition in the AKLT state, while a state like $\lvert +1,0,0,0,-1 \rangle$ is part of it.
We can detect this by composing the AKLT chain with the effects corresponding to such states, and seeing whether the diagram reduces to zero or not.

First, we construct the effects $\langle +1 \rvert$, $\langle -1 \rvert$ and $\langle 0 \rvert$. We encode these spin-1 states on the symmetric subspace of two qubits.

In [ ]:
tikz_spin1_ket0 = r"""\begin{tikzpicture}[tikzfig]
	\begin{pgfonlayer}{nodelayer}
		\node [style=none] (163) at (0, -1.5) {};
		\node [style=Red Node] (165) at (1, -0.75) {$\pi$};
		\node [style=none] (166) at (0, 0) {};
	\end{pgfonlayer}
	\begin{pgfonlayer}{edgelayer}
		\draw (163.center) to (165);
		\draw (166.center) to (165);
	\end{pgfonlayer}
\end{tikzpicture}
"""
spin1_ket0 = zx.Graph.from_tikz(tikz_spin1_ket0)
spin1_ket0.auto_detect_inputs()

spin1_ketplus = ID @ ID.translate(0,0.5)
spin1_ketplus.apply_effect("00")
spin1_ketmin = ID @ ID.translate(0,0.5)
spin1_ketmin.apply_effect("11")
zx.draw(spin1_ket0 @ spin1_ketplus @ spin1_ketmin)
#zx.draw(spin1_ketplus)
#zx.draw(spin1_ketmin)

First, let us construct a state that should *not* be a part of the state, by taking the first and last bra to be equal to $\langle +1 \rvert$.

In [ ]:
effect = spin1_ketplus
for i in range(N-2):
    effect = effect @ spin1_ket0.translate(0,0.5)
effect = effect @ spin1_ketplus.translate(0,0.5)

c = chain + effect
zx.draw(c)

We simplify the diagram with one of the built in simplification routines in PyZX:

In [ ]:
c2 = c.copy()
zx.hsimplify.zh_simp(c2, quiet=False) # Perform the simplification
c2.normalize() # Change the coordinates of the graph for better displaying
zx.draw(c2)

As can be seen, the diagram is completely simplified away, but what it is not showing, is its global scalar correction, which is the important point:

In [ ]:
c2.scalar.to_number()

Let us now see what would happen if instead we postselect for a configuration that *is* part of the AKLT state, such as $\lvert +1,0,0,0,-1 \rangle$:

In [ ]:
effect = spin1_ketplus
for i in range(N-2):
    effect = effect @ spin1_ket0.translate(0,0.5)
effect = effect @ spin1_ketmin.translate(0,0.5)

c = chain + effect
zx.draw(c)

In [ ]:
c2 = c.copy()
zx.hsimplify.zh_simp(c2, quiet=False) # Perform the simplification
c2.normalize() # Change the coordinates of the graph for better displaying
zx.draw(c2)
print(c2.scalar.to_number())